In [5]:
#os.environ['OPENAI_API_KEY'] = <your-api-key>
import openai
import os

api_key = os.getenv('OPENAI_API_KEY')

https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

In [6]:

import pandas as pd
import re
def data_loader():
  """Loads data from CSV files, performs data cleaning and feature engineering.

  Returns:
    pandas.DataFrame: The processed hotel reviews dataset.
  """

  # Read hotel reviews data
  Hotel_Reviews = pd.read_csv("../Data/Hotel_Reviews.csv")

  # Read country data
  countries = pd.read_csv("../Data/countries.csv")

  # Add a "Country" column with NA values
  Hotel_Reviews["Country"] = pd.NA

  # Assign country based on pattern matching in Hotel_Address
  for country_name in countries["name"]:
    pattern = re.compile(country_name, re.IGNORECASE)  # Case-insensitive matching
    Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name

 
  # Convert Review_Date to datetime and extract features
  Hotel_Reviews["date_object"] = pd.to_datetime(Hotel_Reviews["Review_Date"], format="%m/%d/%Y")
  #Hotel_Reviews["time"] = Hotel_Reviews["date_object"].astype(int) / 10**9  # Convert to Unix timestamp
  Hotel_Reviews["month"] = Hotel_Reviews["date_object"].dt.month
  Hotel_Reviews["num_date_object"] = Hotel_Reviews["date_object"].dt.day_of_year / 365  # Normalize by days in a year

  return Hotel_Reviews

# Load the processed data
Hotel_Reviews = data_loader()

C:\Users\alan_\AppData\Local\Temp\ipykernel_30232\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_30232\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_30232\278692378.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  Hotel_Reviews.loc[Hotel_Reviews["Hotel_Address"].str.contains(pattern), "Country"] = country_name
C:\Users\alan_\AppData\Local\Temp\ipykernel_30232\278692378.py:22: UserWarning: This pattern is inte

#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter



# Merge the columns using string concatenation
Hotel_Reviews['MergedColumn_topic'] = (
    Hotel_Reviews['Positive_Review'] + 
    '. ' + Hotel_Reviews['Negative_Review'] 
)
# Select the first 100 rows of the merged column
Hotel_Reviews_UK =Hotel_Reviews[Hotel_Reviews["Country"]== "United Kingdom"]
used_data = Hotel_Reviews_UK['Positive_Review']#Hotel_Reviews['MergedColumn_topic']#[:2000]

# Specify the file name
file_name = "used_data.txt"

# Save the data to a text file
with open(file_name, 'w') as f:
    for line in used_data:
        f.write(line + '\n')

    

In [8]:
docs = used_data.tolist()


Define get_embeddings Function: This function takes a text input, tokenizes it, passes it through the BERT model, and returns the mean of the last hidden state as the text's embedding.
Define Topics and Keywords: It defines a dictionary of topics, each associated with a list of relevant keywords.
Example Texts: It assumes docs[:5] contains example texts to be classified.
Similarity Threshold: A threshold of 0.25 is set to determine if a text is similar enough to a topic.
Classify Texts: For each text:
It computes the embedding of the text.
For each topic, it computes the average embedding of its keywords.
It calculates the cosine similarity between the text's embedding and each topic's average keyword embedding.
It assigns the text to topics where the similarity exceeds the threshold.
Output: It prints the topics under which each text is classified or indicates if no topics match above the threshold

In [4]:
from openai import OpenAI
import os
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)


completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code where logic flows,  \nA curious creature delights and grows,  \nIt dances through functions, a mystical kin,  \nBehold! The magic of recursion begins.\n\nIn a forest of data, so vast and deep,  \nWhere problems like mountains surround us, they creep,  \nA clever solution hides under the trees,  \nCalling itself, like a whispering breeze.\n\n"To solve this conundrum," the wise coder spoke,  \n"I’ll break it in pieces, a plan I invoke!  \nIf I know how to tackle the smaller, I vow,  \nI’ll summon them back in a grand, graceful bow."\n\nSo here comes our function, with purpose so clear,  \nIt checks for a base case, to silence our fear,  \n“A simple condition, when true, will suffice,  \nTo stop my recursion, to end this dice roll of dice.”\n\nBut if not, it calls, as if never to cease,  \nThe same function again, in a dance of release,  \nIt dives into depths, with each layer it peels,  \nLike the rings of a tree, each call reveals.\n\n

In [ ]:
class TextSummarizer:

    # ChatGPT model that we will be using everywhere
    openai_model = "gpt-4o-mini"


    # Constructor    
    def __init__(self):
        self.apikey = self.fetch_api_key()


    # Method to get API key
    def fetch_api_key(self):
        # In here would be all the code required to fetch the api key ...
        return os.environ.get("OPENAI_API_KEY")


    # Method to take list of sentences and return summarized/average single sentence
    def average_sentences(self, list_of_sentences):
        # Instantiate the client
        client = OpenAI(api_key=self.apikey)

        # Write prompt to chatGPT to execute our task
        prompt = "Here is a list of multiple reviews that I want you to summarize and rewrite as a single review that is roughly the same length as the input reviews. The reviews are separated by newline characters \n as follows: {sentences}"
        prompt = prompt.format(sentences = "\n".join(list_of_sentences))

        # Make request to chat GPT
        completion = client.chat.completions.create(
            model=self.openai_model,
            messages=[
                {"role": "system", "content": "You are an assistant that is able to read several reviews and then combine them into a single summarized review. The reviews will be sent to you with a newline character \n separating them. You will return a single review."},
                {"role": "user", "content": prompt}
            ]
        )

        # Get the summarized sentence output from Chat GPT
        summarized_sentence = completion.choices[0].message.content

        # Close the client
        client.close()

        return summarized_sentence

    # Method to summarize a piece of text
    def summarize_text(self, input_text):
        # Instantiate the client
        client = OpenAI(api_key=self.apikey)


        # Make request to chat GPT
        completion = client.chat.completions.create(
            model=self.openai_model,
            messages=[
                {"role": "system", "content": "You are an assistant that is able to read a piece of text and summarize it. Your summary will be 4 sentences or less."},
                {"role": "user", "content": "Here is the text to be summarized below the newline character.\n {text}".format(text=input_text)}
            ]
        )

        # Get the summary of the inptu text output from Chat GPT
        summary_output = completion.choices[0].message.content

        # Close the client
        client.close()

        return summary_output        
    
    
    # Method to summarize a piece of text
    def summarize_text_topic(self, input_text):
        # Instantiate the client
        client = OpenAI(api_key=self.apikey)


        # Make request to chat GPT
        completion = client.chat.completions.create(
            model=self.openai_model,
            messages=[
                {"role": "system", "content": """You are an assistant that is able to read a piece of text and summarize it. Please provide a one sentence general summary.
                 Additionally you will write  a summary sentence on each of the for topics: Room, Food and Drinks, Location, Internet and Work and Surprise.
                 Here are some keywords for each topic 
                "Room": ["room", "rooms", "upgrade", "clean", "tidy", "large", "bathroom", "bed", "TV", "shower"] all aspects of describing the status of the room,
                "Food and Drinks": ["drinks", "cocktails", "bottle", "breakfirst", "dinner", "menu", "caffee", "tee", "delicious", "continental", "waiter","restaurant "] all aspects describing the quality of food like breakfirst and bar,
                "Location": ["close", "far", "next", "park", "train", "bicicle", "car", "walk", "tee", "building", "neighborhood", "cab service", "airport", "subway", "stairs"] all aspects describing the location, surrounding and connection of the hotel,
                "Internet and Work": ["wifi", "Internet", "connection", "work", "password", "computer", "meeting", "signal"] all aspects describing abilty to work from the hotel with a focus on internet connection,
                "Surprise": ["everything", "honestly", "surprising", "change", "unfortunately", "refund"] all aspects which are supringly and not expected by the reviewer,
                Feel free to say that the reviews do not specifically address certain topics.
                 """},
                {"role": "user", "content": "Here is the text to be summarized below the newline character.\n {text}".format(text=input_text)}
            ]
        )

        # Get the summary of the inptu text output from Chat GPT
        summary_output = completion.choices[0].message.content

        # Close the client
        client.close()

        return summary_output    




In [12]:
gpt_summarizer = TextSummarizer()
avg_sentence = gpt_summarizer.average_sentences(docs[:5])

print(avg_sentence)

This hotel offers a very comfortable stay with cozy beds and pleasant facilities, including a lovely garden that’s perfect for summer evenings. The staff is friendly and exceptionally helpful, particularly in organizing daytime activities. Located in a quiet area, it provides great access to public transportation, making it easy to explore nearby restaurants and shops. While the bathrooms could use some ventilation, and the rooms are on the smaller side, the overall experience is enjoyable. The hotel also serves a delightful breakfast and has a laundrette conveniently located nearby for longer stays. Overall, it’s a great value for London, and I would definitely recommend staying here again.


In [14]:
summarized_output_text = gpt_summarizer.summarize_text(docs[:5])
summarized_output_text 

"The hotel is praised for its comfortable beds, friendly staff, and lovely garden facilities, with a quiet location that's close to public transport and local eateries. Guests appreciate the helpfulness of the staff and the bonus of a laundrette nearby for long-term travelers. While some noted the older facilities and small bathrooms, the overall experience is seen as a great value for London, with many indicating they would stay again."

In [18]:
gpt_summarizer = TextSummarizer()
summarized_output_text = gpt_summarizer.summarize_text_topic(docs[:5])
print(summarized_output_text) 

Overall, the reviews highlight a comfortable stay with friendly staff, a well-located hotel near public transport and restaurants, and some minor issues with room size and bathroom ventilation.

**Room:** The rooms are described as small but clean and comfortable, with good showers, although the bathrooms may lack ventilation.

**Food and Drinks:** The hotel offers a good breakfast and is near many restaurants, providing a variety of dining options within walking distance.

**Location:** The hotel is in a quiet area with great access to public transport, particularly near the subway, making it convenient for exploring the city and reaching the airport.

**Internet and Work:** The reviews do not specifically address internet connectivity or work facilities, suggesting it may not be a focus for guests.

**Surprise:** Reviews indicate positive surprises with the property, such as lovely facilities like the garden, friendly and accommodating staff, and overall good value for the London are